In [1]:
import pandas as pd
import numpy as np
import itertools
import torch
import torch.nn as nn
import nltk
import spacy
import re
import os

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig,BertModel
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nlp = spacy.load("en_core_web_sm")
# download necessary resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Set TOKENIZERS_PARALLELISM to false
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


# Preprocess And Tokenizer

In [3]:
# data = pd.read_csv("/kaggle/input/10kdataset-corrected/10krowsAttribute_corrected.csv")
data = pd.read_csv("/kaggle/input/10kdataset-corrected/10krowsAttributeMissing_words.csv")

data.head()
data.shape

(10000, 5)

In [4]:
# String to add
string_to_add = " <EOS>"

# Add the string to each element in 'col1' using concatenation
data['rule_attribute'] = data['rule_attribute'] + string_to_add

In [5]:
data.columns

Index(['sentence', 'word_labels', 'rule_attribute', 'missing_words',
       'attr_words'],
      dtype='object')

In [6]:
df_ = data[['sentence', 'word_labels', 'rule_attribute', 'missing_words','attr_words']].drop_duplicates().reset_index(drop=True)
df_.head()

,sentence,word_labels,rule_attribute,missing_words,attr_words
0,The Senate granted Bibulus a thanksgiving of t...,"B-ent,I-ent,O,O,O,B-attr,O,B-qty,I-qty",duration thanksgiving <EOS>,duration <EOS>,['thanksgiving']
1,Takshang is 30.7 km from Gangtok and the popul...,"B-ent,O,B-qty,I-qty,B-attr,I-attr,O,O,O,O,O,O,...",distance from Gangtok <EOS>,distance <EOS>,"['from', 'Gangtok']"
2,The rocky banks of the Nugush and Belaya rives...,"O,B-attr,I-attr,O,B-ent,I-ent,I-ent,I-ent,O,B-...",rocky banks height <EOS>,<EOS>,"['rocky', 'banks', 'reach', 'height']"
3,The Padayani was presented for 8 days from Med...,"O,B-ent,O,B-attr,O,B-qty,I-qty,O,O,O,O,O",duration presented <EOS>,duration <EOS>,['presented']
4,Average rainfall in Windsor Locks is 46.27 inches,"O,B-attr,O,B-ent,I-ent,O,B-qty,I-qty",average rainfall <EOS>,average <EOS>,['rainfall']


In [7]:
df_.columns=['sentence','word_labels','rule_attribute','missing_words','attr_words']
df_.columns

Index(['sentence', 'word_labels', 'rule_attribute', 'missing_words',
       'attr_words'],
      dtype='object')

In [8]:
Tag = ['O','B-ent','I-ent','B-attr','I-attr','B-qty','I-qty']
labels_to_ids = {k: v for v, k in enumerate(Tag)}
ids_to_labels = {v: k for v, k in enumerate(Tag)}
labels_to_ids

{'O': 0,
 'B-ent': 1,
 'I-ent': 2,
 'B-attr': 3,
 'I-attr': 4,
 'B-qty': 5,
 'I-qty': 6}

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Let's see how to increase the vocabulary of Bert model and tokenizer
# tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")
# model = BertModel.from_pretrained("google-bert/bert-base-uncased")

num_added_toks = tokenizer.add_tokens(["<EOS>","<SOS>"])
print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
# model.resize_token_embeddings(len(tokenizer))
len(tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

We have added 2 tokens


30524

In [10]:
# convert_tokens_to_ids
tokenizer.convert_tokens_to_ids(["<EOS>"]),tokenizer.convert_tokens_to_ids(["<SOS>"])

([30522], [30523])

In [11]:
sent="The regional offices in Europe and North America are 80 percent owned by TBWA and 20 percent owned by Hakuhodo ; the one in Japan is 80 percent owned by Hakuhodo and 20 percent owned by TBWA".split()
tokenizer(sent,is_split_into_words=True,max_length=64,add_special_tokens=True,truncation=True)

{'input_ids': [101, 1996, 3164, 4822, 1999, 2885, 1998, 2167, 2637, 2024, 3770, 3867, 3079, 2011, 26419, 4213, 1998, 2322, 3867, 3079, 2011, 5292, 5283, 6806, 3527, 1025, 1996, 2028, 1999, 2900, 2003, 3770, 3867, 3079, 2011, 5292, 5283, 6806, 3527, 1998, 2322, 3867, 3079, 2011, 26419, 4213, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# Encoder Decoder

In [12]:
################################## ENCODER - DECODER DATA LOADER ######################################

class EncoderDecoderDataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len_encoder,max_len_decoder):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len_encoder = max_len_encoder
        self.max_len_decoder = max_len_decoder

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")
#         rule_attribute = self.data.rule_attribute[index].strip().split()
        rule_attribute = self.data.missing_words[index].strip().split()

        # print(sentence)
        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                            # is_pretokenized=True,
                            is_split_into_words=True,
                            return_offsets_mapping=True,
                            padding='max_length',
                            truncation=True,
                            max_length=self.max_len_encoder)

        decoding = self.tokenizer(rule_attribute,
                            # is_pretokenized=True,
                            is_split_into_words=True,
                            return_offsets_mapping=True,
                            padding='max_length',
                            truncation=True,
                            max_length=self.max_len_decoder,
                            add_special_tokens= False)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]

        encode_item = {key: torch.as_tensor(val) for key, val in encoding.items()}

        encoded_labels=[]
        label_index = 0
#         new=[1:encoding["offset_mapping"]]
        for idx,off_map in enumerate(encoding["offset_mapping"]):
#             print(off_map)
            start, end= off_map
            if start == 0 and end == 0:
                encoded_labels.append(-100)
            elif start==0 and end!=0:
                # Otherwise, append the corresponding label to encoded_labels
                encoded_labels.append(labels[label_index])
#                 print(encoding["offset_mapping"][idx+1][])
                if encoding["offset_mapping"][idx+1][0]==0 and encoding["offset_mapping"][idx+1][1]!=0:
                    label_index += 1
                 # Move to the next label
            elif start!=0 and end!=0:
                encoded_labels.append(labels[label_index])
                if encoding["offset_mapping"][idx+1][0]==0 and encoding["offset_mapping"][idx+1][1]!=0:
                    label_index += 1

        # step 4: turn everything into PyTorch tensors
        encode_item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        encode_item['labels'] = torch.as_tensor(encoded_labels)
        encode_item['qty_pivot'] = torch.logical_or(encode_item['labels']==5,encode_item['labels']==6).int()


        decode_item = {key: torch.as_tensor(val) for key, val in decoding.items()}
#         decode_item["rule"]=rule_attribute
        return (encode_item,decode_item)
############################### ENCODER DATA LOADER ####################
  def __len__(self):
        return self.len

In [13]:
train_size = 0.95
train_dataset = df_.sample(frac=train_size,random_state=200)
test_dataset = df_.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df_.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = EncoderDecoderDataset(df_, tokenizer, 64,30)
testing_set = EncoderDecoderDataset(test_dataset, tokenizer, 64,10)

FULL Dataset: (10000, 5)
TRAIN Dataset: (9500, 5)
TEST Dataset: (500, 5)


# BERT CLASSIFIER CREATION

# Inference Function

In [25]:
import torch.nn.functional as F
def get_values_before_first_target(tensor, target):
    try:
        tensor_list = tensor.tolist()  # Convert tensor to list
        index_target = tensor_list.index(target)
        return tensor_list[:index_target]
    except ValueError:
        print("Target value not found in the tensor.")
        return tensor_list

df_test = pd.read_csv("/kaggle/input/testdataqtycorrected/100testrowsqty_corrected.csv")
print(df_test.columns)
df_test.drop("Unnamed: 0",inplace=True,axis=1)
df_test.columns = ["sentence","word_labels","rule_attribute"]

def get_encoder_preds(offset_map,flattened_tags):

    # Convert logits to predictions (based on argmax or thresholding)
#     predictions = torch.argmax(logits, dim=-1)
    predicted_labels=[]
    label_index = 0
#         new=[1:encoding["offset_mapping"]]
    count=-1
    for idx_off,off_map in enumerate(offset_map[0]):
        # print(off_map)
        start, end= off_map
#             print("label_index",label_index," ", (start, end))
        # If start and end offsets are both zero, append zero to encoded_labels
        if start == 0 and end == 0:
            # predicted_labels.append(predictions[0][idx].item())
            # actual_labels.append(-100)
            count+=1
        elif start==0 and end!=0:
            # Otherwise, append the corresponding label to encoded_labels
            predicted_labels.append(ids_to_labels[flattened_tags[idx_off].item()])
#             actual_labels.append(ids_to_labels[labels[0][idx_off].item()])

            # predicted_labels.append(predictions[0][idx_off].item())
            # actual_labels.append(labels[0][idx_off].item())
        if count==1:
              break
#     print(f"Sentence: {df_test.sentence[idx]}")
#     print(f"Predicted_labels: {predicted_labels}")
    return predicted_labels

#     actual_list.append(actual_labels)
#     sentences.append(df_test.sentence[idx])

def encoder_inference(sentence,predsLabels):
#     predsLabels = df_pred['Predicted Labels'].values
#     sentences = df_pred['Sentence'].values
#     attribute_list=[]
#     for idx in range(len(sentences)):
    attr_words=[]
    
    preds = predsLabels
#     print(len(sentence.split()),len(preds))
    
#     print(f"Encoder Inference preds: {preds}")
    for idx_word,word in enumerate(sentence.split()):
        # print(predsLabels[idx][1:-1].split(","))
        # print(word)
        value = preds[idx_word]
        if value=="B-attr" or value=="I-attr":
            attr_words.append(word)
      # print(attr_words)
#       attribute_list.append(" ".join(attr_words))
    return " ".join(attr_words)


def inference_function_merge(df_test,max_len_encoder,max_len_decoder):
    pred_list,label_list,sent_list,boolean = [],[],[],[]
    model.eval()
    test_set = Dataset_inference(df_test, tokenizer, max_len_encoder = max_len_encoder, max_len_decoder=max_len_decoder)

    test_params = {'batch_size': 1,
                        'shuffle': False,
                        'num_workers': 1
                        }
    testing_loader = DataLoader(test_set, **test_params)
    labels_list = []
    # for sentence,word_labels,rule_attr in zip(data.sentence,data.labels,data.rule_attri):
    for idx, batch in enumerate(testing_loader):
#         df.iloc[2]['B']
        sentence = df_test.iloc[idx]['sentence']
#         print(f"Sentnece: {sentence}")

        encode_ids = batch[0]['input_ids'].to(device, dtype = torch.long)
        encode_mask = batch[0]['attention_mask'].to(device, dtype = torch.long)
        # print(f"Mask Size: {mask.size()}")
        labels = batch[0]['labels'].to(device, dtype = torch.long)
        qty_pivot = batch[0]['qty_pivot'].to(device, dtype = torch.long)
        offset_map = batch[0]['offset_mapping'].to(device, dtype = torch.long)
        decode_ids = batch[1]['input_ids'].to(device, dtype = torch.long)
        decode_mask = batch[1]['attention_mask'].to(device, dtype = torch.long)


        with torch.no_grad():

            # outputs = model(input_ids=encode_ids, attention_mask=encode_mask,target_ids= decode_ids,qty_pivot =qty_pivot,labels=labels)
            # print(outputs.size())
            encoder_outputs,encoder_logits = model.encode(input_ids=encode_ids, attention_mask=encode_mask,qty_pivot=qty_pivot,labels=labels)

            ########### NEWLY ADDED STUFFS #####################
            # indices_list = find_indices_of_value(encode_ids, value=102)
            active_tags = encoder_logits.view(-1, 7)
            flattened_tags = torch.argmax(active_tags, axis=1) # shape (batch_size * seq_len,)
#             print(f"tags:{flattened_tags}")
            
            predLabels = get_encoder_preds(offset_map,flattened_tags)
            enc_attribute = encoder_inference(sentence,predLabels)
            
            # attr_tags = torch.logical_or(flattened_tags==3,flattened_tags==4).view(-1,input_ids.size()[1]).type(torch.LongTensor).to(device).unsqueeze(2)
#             query = create_query(encode_ids,flattened_tags,decode_ids)
            # print(f"Query in Validation: {query}")

            # memory = model.encode(src, src_mask)
            ys,preds = (torch.ones(1, 1)*30523).to(device),[]

            for i in range(max_len_decoder):

                decoded_outputs = model.decode(input_ids=encode_ids,labels=flattened_tags,encoder_outputs=encoder_outputs,encoder_logits=encoder_logits,target_ids=ys) #whole query
                # log_softmax(self.proj(x), dim=-1)
                prob = model.generate(decoded_outputs)
                # print(f"Prob dim:  {prob.size()}")

                next_word_probs = torch.softmax(prob, dim=-1)
#                 next_word_probs = F.log_softmax(prob, dim=-1)

                # print(f"next_word_probs dim:  {next_word_probs.size()}")

                _, next_word_index = torch.max(next_word_probs, dim=-1)
                # print(f"next word index: {next_word_index.size()}")
                if next_word_index[:,i]==30522:
                    break
                next_word = tokenizer.convert_ids_to_tokens(next_word_index[:,i])

                # _, next_word = torch.max(prob, dim=2)
                # print(f"next word: {next_word}")
                # print(f"next word dim: {next_word.size()}")

                # next_word = tokenizer.convert_ids_to_tokens(next_word)
                # print(f"next word: {next_word}")
                # print(f"next_word_index.item(): {next_word_index[:,i]}")

                # Convert next_word_index tensor to the device before concatenating
                next_word_index_tensor = torch.ones(1, 1, device=device) * next_word_index[:, i]

                # Concatenate tensors
                preds.append(next_word[0])
                ys = torch.cat([ys, next_word_index_tensor], dim=1)
            # print("Example Untrained Model Prediction:", ys,preds)
            
            pred_list.append(" ".join(preds)+" "+enc_attribute)
            labels_list.append(predLabels)

    return pred_list,labels_list

Index(['Unnamed: 0', 'sentence', 'word_labels', 'rule_attribute'], dtype='object')


In [15]:
################################## ENCODER - DECODER DATA LOADER ######################################

class Dataset_inference(Dataset):
        def __init__(self, dataframe, tokenizer, max_len_encoder,max_len_decoder):
            self.len = len(dataframe)
            self.data = dataframe
            self.tokenizer = tokenizer
            self.max_len_encoder = max_len_encoder
            self.max_len_decoder = max_len_decoder

        def __getitem__(self, index):
            # step 1: get the sentence and word labels
            sentence = self.data.sentence[index].strip().split()
            word_labels = self.data.word_labels[index].split(",")
            rule_attribute = self.data.rule_attribute[index].strip().split()
    #         rule_attribute = self.data.missing_words[index].strip().split()

            # print(sentence)
            # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
            # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
            encoding = self.tokenizer(sentence,
                                # is_pretokenized=True,
                                is_split_into_words=True,
                                return_offsets_mapping=True,
                                padding='max_length',
                                truncation=True,
                                max_length=self.max_len_encoder)

            decoding = self.tokenizer(rule_attribute,
                                # is_pretokenized=True,
                                is_split_into_words=True,
                                return_offsets_mapping=True,
                                padding='max_length',
                                truncation=True,
                                max_length=self.max_len_decoder,
                                add_special_tokens= False)

            # step 3: create token labels only for first word pieces of each tokenized word
            labels = [labels_to_ids[label] for label in word_labels]

            encode_item = {key: torch.as_tensor(val) for key, val in encoding.items()}

            encoded_labels=[]
            label_index = 0
    #         new=[1:encoding["offset_mapping"]]
            for idx,off_map in enumerate(encoding["offset_mapping"]):
    #             print(off_map)
                start, end= off_map
                if start == 0 and end == 0:
                    encoded_labels.append(-100)
                elif start==0 and end!=0:
                    # Otherwise, append the corresponding label to encoded_labels
                    encoded_labels.append(labels[label_index])
    #                 print(encoding["offset_mapping"][idx+1][])
                    if encoding["offset_mapping"][idx+1][0]==0 and encoding["offset_mapping"][idx+1][1]!=0:
                        label_index += 1
                     # Move to the next label
                elif start!=0 and end!=0:
                    encoded_labels.append(labels[label_index])
                    if encoding["offset_mapping"][idx+1][0]==0 and encoding["offset_mapping"][idx+1][1]!=0:
                        label_index += 1

            # step 4: turn everything into PyTorch tensors
            encode_item = {key: torch.as_tensor(val) for key, val in encoding.items()}
            encode_item['labels'] = torch.as_tensor(encoded_labels)
            encode_item['qty_pivot'] = torch.logical_or(encode_item['labels']==5,encode_item['labels']==6).int()


            decode_item = {key: torch.as_tensor(val) for key, val in decoding.items()}
    #         decode_item["rule"]=rule_attribute
            return (encode_item,decode_item)
        ############################### ENCODER DATA LOADER ####################
        def __len__(self):
            return self.len

# Utilities

In [16]:

def preprocess_attr(text):
  # lowercase the text
    text = str(text).lower().strip()
    '''text=text.replace("'s","")text=text.replace(" 's","")'''

# tokenize the text
    tokens = nltk.word_tokenize(text)
    for i in range(len(tokens)):
        w=tokens[i]

        w=w.replace(" ","")
        tokens[i]=w

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
  #  print(filtered_tokens)
    # remove special characters using regex
    preprocessed_text = ' '.join(tokens)
    # process the text with spaCy
    doc = nlp(preprocessed_text)

  # lemmatize each token and join them back into a string
    #SpaCy lemmatizer
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    text = re.sub('[^a-zA-Z0-9\s]', '',  lemmatized_text)
    #text=re.sub('[^a-zA-Z0-9\s]', '',  preprocessed_text)
    tokens = nltk.word_tokenize(text)
    return tokens

def relaxed_precision(pred, gt):
    # split predicted and ground truth into words
    # calculate overlap
    overlap = len(set(pred).intersection(set(gt)))

    # calculate relaxed precision
    if(len(pred)!=0):
        rp = overlap / len(pred)
    else:
        rp=0
    return rp

def relaxed_recall(pred, gt):
    # calculate overlap
    overlap = len(set(pred).intersection(set(gt)))

    if(len(gt)!=0):
    # calculate relaxed recall
        rr = overlap / len(gt)
    else:
        rr=0

    return rr


df_decoder = pd.read_csv("/kaggle/input/testdataqtycorrected/100testrowsqty_corrected.csv")
print(df_decoder.columns)
# decoder_attr = df_decoder["predictedAttribute"].values
ground_truth_attr = df_decoder["rule_attribute"].values
sent = df_decoder["sentence"].values


def check_F1_score(ground_truth_attr,decoder_attr):
    relaxed_precisions = []
    relaxed_recalls = []

    for i in range(len(ground_truth_attr)):
        rp = relaxed_precision(preprocess_attr(decoder_attr[i]),preprocess_attr(ground_truth_attr[i]))
        rr = relaxed_recall(preprocess_attr(decoder_attr[i]),preprocess_attr(ground_truth_attr[i]))
        relaxed_precisions.append(rp)
        relaxed_recalls.append(rr)

    # calculate average relaxed precision and recall
    avg_relaxed_precision = np.mean(relaxed_precisions)
    avg_relaxed_recall = np.mean(relaxed_recalls)
    # print(f"Relaxed precision for attribute : {avg_relaxed_precision}")
    # print(f"Relaxed recall for attribute: {avg_relaxed_recall}")
    f1score = (2*avg_relaxed_recall*avg_relaxed_precision)/(avg_relaxed_precision+avg_relaxed_recall)
#     print(f"f1 score for attribute: {f1score}")
    return f1score,decoder_attr

Index(['Unnamed: 0', 'sentence', 'word_labels', 'rule_attribute'], dtype='object')


In [17]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(training_loader,alpha=0.5,MAX_GRAD_NORM=3):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training model
    model.train()

    for idx, batch in enumerate(training_loader):

        encode_ids = batch[0]['input_ids'].to(device, dtype = torch.long)
        mask = batch[0]['attention_mask'].to(device, dtype = torch.long)
        # print(f"Mask Size: {mask.size()}")
        qty_pivot = batch[0]['qty_pivot'].to(device, dtype = torch.long)
        labels = batch[0]['labels'].to(device, dtype = torch.long)
        decode_ids = batch[1]['input_ids'].to(device, dtype = torch.long)
        decode_mask = batch[1]['attention_mask'].to(device, dtype = torch.long)


        # print(ids.size(),mask.size(),labels.size())
        # outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        outputs = model(input_ids=encode_ids, attention_mask=mask,qty_pivot=qty_pivot,labels=labels,target_ids= decode_ids)

        decoded_outputs,hidden_states,enc_logits = outputs[0],outputs[1],outputs[2]

        logits_decoder = model.generate(decoded_outputs)
        # print(f"Logits decoder: {logits_decoder.size()}")

        logits_flat = logits_decoder[:, 0:].contiguous().view(-1, logits_decoder.size(-1))
        decoder_target_ids_flat = decode_ids.contiguous().view(-1)

        enc_logits_flat = enc_logits[:,0:].contiguous().view(-1,enc_logits.size(-1))
        enc_flat_true =  labels.contiguous().view(-1)
        # # Cross-Entropy Loss
        # print(f"Logits Flat Size: {logits_flat}")
        # print(f"Shifted Input Ids Size: {shifted_input_ids_flat}")
        # print(f"Shifted Input Ids: {shifted_input_ids_flat}")
        # print(f"Logits Flat Values: {logits_flat}")
        loss_fn_dec = nn.CrossEntropyLoss(ignore_index=0,label_smoothing=0.1)  # Specify the pad_token_id
        loss_fn_enc = nn.CrossEntropyLoss(ignore_index=-100)
        initial_loss_enc = loss_fn_enc(enc_logits_flat,enc_flat_true)
        initial_loss_dec = loss_fn_dec(logits_flat, decoder_target_ids_flat)
        # print(outputs.size())
        # initial_loss,tr_logits = outputs[0],outputs[1]

        # print(tr_logits.size(),"Logits Size")
        # print("LOss:",loss, "Logits:",tr_logits)

        # initial_loss
        # print(initial_loss.item(),"\n",logits)
#         alpha=0.45
        alpha=alpha
        total_loss = (1-alpha)*initial_loss_enc+ alpha*initial_loss_dec
        tr_loss+= (1-alpha)*initial_loss_enc.item()+ alpha*initial_loss_dec.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            # print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = decode_ids.view(-1) # shape (batch_size * seq_len,)
        active_logits = logits_decoder.view(-1, 30524) # shape (batch_size * seq_len, num_labels)

        # print("Active logits",active_logits.shape)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # print("Flattened predictions",flattened_predictions)
        # print("Flattened predictions Shape",flattened_predictions.size())
        # print("Flattened Targets",flattened_targets)
        # print("Flattened Targets Shape",flattened_targets.size())

        # print("Labels",labels.shape)
        # only compute accuracy at active labels
        active_accuracy = flattened_targets.view(-1)!=0 # shape (batch_size, seq_len)
        # print(active_accuracy,"Active Accuracy")
        # active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        # print("Active accuracy",active_accuracy)

        labels = torch.masked_select(flattened_targets, active_accuracy)
        # print(f"Active labels: {labels}")
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        # print(f"Predicted labels: {predictions}")

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")



# BERT Classifier

In [18]:
from transformers.models.bert.modeling_bert import BertEncoder, BertPooler, BertEmbeddings, BaseModelOutputWithPoolingAndCrossAttentions
from transformers import BertConfig, BertModel
import torch
from torch import nn

# https://discuss.huggingface.co/t/how-to-use-additional-input-features-for-ner/4364/26

class BertEmbeddingsV2(BertEmbeddings):
    def __init__(self, config):
        super().__init__(config)
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)
        max_number_of_pos_tags = 2
        self.pos_tag_embeddings = nn.Embedding(max_number_of_pos_tags, config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # position_ids (1, len position emb) is contiguous in memory and exported when serialized
        self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)))
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")

    def forward(self, input_ids=None, qty_pivot=None, token_type_ids=None, position_ids=None, inputs_embeds=None, past_key_values_length=0):
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        seq_length = input_shape[1]

        if position_ids is None:
            position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]

        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)

        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        # print(f"Qty Pivot: {qty_pivot}")
        pos_tag_embeddings = self.pos_tag_embeddings(qty_pivot)

        embeddings = inputs_embeds + token_type_embeddings + pos_tag_embeddings
        if self.position_embedding_type == "absolute":
            position_embeddings = self.position_embeddings(position_ids)
            embeddings += position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

class BertModelV2(BertModel):
    """
    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in `Attention is
    all you need <https://arxiv.org/abs/1706.03762>`__ by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit,
    Llion Jones, Aidan N. Gomez, Lukasz Kaiser and Illia Polosukhin.
    To behave as an decoder the model needs to be initialized with the :obj:`is_decoder` argument of the configuration
    set to :obj:`True`. To be used in a Seq2Seq model, the model needs to initialized with both :obj:`is_decoder`
    argument and :obj:`add_cross_attention` set to :obj:`True`; an :obj:`encoder_hidden_states` is then expected as an
    input to the forward pass.
    """

    def __init__(self, config, add_pooling_layer=False):
        super().__init__(config)
        self.config = config

        self.embeddings = BertEmbeddingsV2(config)
        self.encoder = BertEncoder(config)

        self.pooler = BertPooler(config) if add_pooling_layer else None

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        # pos_tag_ids=None,
        qty_pivot = None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=True,
        return_dict=None,
    ):
        """
        encoder_hidden_states  (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`, `optional`):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in ``[0, 1]``:
            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (:obj:`tuple(tuple(torch.FloatTensor))` of length :obj:`config.n_layers` with each tuple having 4 tensors of shape :obj:`(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
            If :obj:`past_key_values` are used, the user can optionally input only the last :obj:`decoder_input_ids`
            (those that don't have their past key value states given to this model) of shape :obj:`(batch_size, 1)`
            instead of all :obj:`decoder_input_ids` of shape :obj:`(batch_size, sequence_length)`.
        use_cache (:obj:`bool`, `optional`):
            If set to :obj:`True`, :obj:`past_key_values` key value states are returned and can be used to speed up
            decoding (see :obj:`past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
            batch_size, seq_length = input_shape
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
            batch_size, seq_length = input_shape
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)
        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        # print(f"Define V2 Qty_pivot: {qty_pivot.size()}")

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            # pos_tag_ids=pos_tag_ids,
            qty_pivot = qty_pivot,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        encoder_outputs = self.encoder(
            embedding_output,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

#         print(f"Encoder first: {encoder_outputs[1:][0][1].size()}")
#         print(f"Encoder Output : {len(encoder_outputs[1:][0])}")
#         print(f"Hidden States: {len(encoder_outputs.hidden_states)}")
#         print(f"Encoder zero: {encoder_outputs[1:][0][0].size()}")
#         print(f"Hiden State 1st: {encoder_outputs.hidden_states[0][0].size()}")
#         print(f"Hiden State 2nd: {encoder_outputs.hidden_states[1][0].size()}")

        '''
        Encoder first: torch.Size([4, 128, 768])
        Encoder Output : 13      ---------- this is what we take forward
        Hidden States: 13
        Encoder zero: torch.Size([4, 128, 768])
        Hiden State 1st: torch.Size([128, 768])
        Hiden State 2nd: torch.Size([128, 768])
        '''
        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


In [19]:

# Create the BertClassfier class
class BertClassifier_1(nn.Module):
    """
    Bert Model for Classification Tasks.
    """
    def __init__(self, tokenizer, freeze_bert=False,num_labels=len(labels_to_ids),drop1=0.1,drop2=0.1,activation="relu"):

        super(BertClassifier_1, self).__init__()

        # Instantiate BERT model
        # self.embedding_1 =  nn.Embedding(n, d, max_norm=False)
        self.num_labels=num_labels
        # self.bert = BertModel.from_pretrained('bert-base-uncased',add_pooling_layer=False)
        self.activation = activation
        config = BertConfig()
        self.bertV2 = BertModelV2.from_pretrained("bert-base-uncased", config=config)
        self.bertV2.resize_token_embeddings(len(tokenizer))
        self.enc_modules = [self.bertV2.embeddings , self.bertV2.encoder.layer[:10]]
#         self.enc_modules = [self.bertV2.encoder.layer[:10]]

        # D_in, H, D_out = 768, 264, self.num_labels

        D_in,H_1,H_2,H_3,D_out = 768, 512,264,128,self.num_labels

#         self.fc1 = nn.Dropout(drop1)
        self.fc2 = nn.Linear(D_in,H_1)#768x512
        self.fc3 = nn.Dropout(drop1)
        self.fc4 = nn.Linear(H_1,H_2)#512x264
        self.fc5 = nn.Dropout(drop2)
        self.fc6 = nn.Linear(H_2,D_out)
#         self.fc7 = nn.Linear(H_3,D_out)#128x7

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.gelu = nn.GELU()

        # count = 0
        # for param in self.bertV2.parameters():
        #     count+=1
        #     if count<=165:
        #       param.requires_grad = False

        for module in self.enc_modules:
            for param in module.parameters():
                param.requires_grad = False

#         for name, param in self.bertV2.named_parameters():
#           if name=="embeddings.pos_tag_embeddings.weight":
# #             print("Worked")
#             param.requires_grad=True

#         for name, param in self.bertV2.named_parameters():
# #           if name=="bertV2.embeddings.pos_tag_embeddings.weight":
#             print(name,param.requires_grad)
#         # self.bertV2.embeddings.pos_tag_embeddings.parameters =  True

    def forward(self, input_ids, attention_mask,qty_pivot,labels):
      ## ADD MASK LABELS as well here, positioning or something,
      # def forward(self, encoding):

        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """

        outputs = self.bertV2(input_ids=input_ids,attention_mask=attention_mask,qty_pivot=qty_pivot)
        # print("Last Hidden State size: ", outputs['last_hidden_state'].size())
        ## find the output where the pivoting for masking is done from the dataset, send that to get the MASK prediction into one of the labels
#         self.labels=labels

        if self.activation=="relu":
#             x = self.fc1(outputs.last_hidden_state)
            x = self.relu(outputs.last_hidden_state)
            x = self.fc2(x)
            x = self.relu(x)
            x = self.fc3(x)
            x = self.fc4(x)
            x = self.relu(x)
            x = self.fc5(x)

            logits = self.fc6(x)



        else:
            x = self.fc1(outputs.last_hidden_state)
            x = self.fc2(x)
            x = self.tanh(x)
            x = self.fc3(x)
            logits = self.fc4(x)


        return outputs,logits

# model = BertClassifier_1(tokenizer=tokenizer,freeze_bert=True)
# model.to(device)

# SEQ2SEQ MODEL

**When it was providing answers it was because the fact that answer was already given to it in the query.** <br>
**So we generate the answers from the Tagging thing and feed it to the decoder as QUERY** <br>
**Thus helping the model to see what it needs to see. Also, it can be the point that encoder is more weighed upon then the decoder.**

* Few Experiments:
1. Use the general one currently trying with only 8 decoder layers.<br> <br>
2. Use the one with Query from the Encoder output tags, enforcing the important attributes. <br> Play with the weights of loss.<br>
**Remember to put **is_causal=False**<br>
3. Train the Encoder alone, see what can be done about it.<br> <br>
4. Use the trained encoder to give the decoder the query output which are tagged as attribute. along with CLS and not with CLS.

In [20]:
# import torch

def apply_mask_to_encoder_hidden_state(encoder_hidden_state, attribute_tags, indices):
    # Apply the mask
    masked_tensor = torch.zeros_like(attribute_tags)
    for i, row in enumerate(attribute_tags):
        indices_to_mask = indices[i]
#         print(f"row value: {row}")
        masked_tensor[i, :indices_to_mask] = row[:indices_to_mask]

    # Expand the dimensions of mask_tensor to match the shape of encoder_hidden_state
    expanded_mask = masked_tensor.unsqueeze(-1).expand_as(encoder_hidden_state)

    # Apply the mask
    masked_encoder_hidden_state = encoder_hidden_state * expanded_mask

    return masked_encoder_hidden_state


def find_indices_of_value(input_ids, value=102):
    indices_list = []
    for row in input_ids:
        indices = torch.where(row == value)[0]
        indices_list.append(indices.item() if len(indices) > 0 else None)
#     print(f"Indices function: {indices_list}")
    return indices_list


In [22]:
import torch
import torch.nn as nn
from transformers import BertModel, BertConfig

def apply_mask_to_encoder_hidden_state(encoder_hidden_state, attribute_tags, indices):
    # Apply the mask
    masked_tensor = torch.zeros_like(attribute_tags)
    for i, row in enumerate(attribute_tags):
        indices_to_mask = indices[i]
#         print(f"row value: {row}")
        masked_tensor[i, :indices_to_mask] = row[:indices_to_mask]

    # Expand the dimensions of mask_tensor to match the shape of encoder_hidden_state
    expanded_mask = masked_tensor.unsqueeze(-1).expand_as(encoder_hidden_state)

    # Apply the mask
    masked_encoder_hidden_state = encoder_hidden_state * expanded_mask

    return masked_encoder_hidden_state


def find_indices_of_value(input_ids, value=102):
    indices_list = []
    for row in input_ids:
        indices = torch.where(row == value)[0]
        indices_list.append(indices.item() if len(indices) > 0 else None)
#     print(f"Indices function: {indices_list}")
    return indices_list


def create_query(input_ids,tags,target_ids):
    ## Size should be (4x128)
    # Mask for tensor_1 where parts with 0 are masked
    mask_tensor_1_zero = input_ids != 0
    # print(mask_tensor_1_zero)
    # Mask for tensor_2 where parts with values 3 and 4 are masked
    mask_tensor_2_3_4 = (tags == 3) | (tags == 4)
    double_mask = mask_tensor_1_zero & mask_tensor_2_3_4

    # get the inputs_ids that satisfy both the condition
    result_tensor = double_mask.float() * input_ids

    # Get indices of non-zero elements in each row
    nonzero_indices = torch.nonzero(result_tensor, as_tuple=False)

    # Create a zero tensor of the same size as the original tensor
    query = torch.zeros_like(result_tensor)

    # Fill the result tensor with the corresponding non-zero values from the original tensor
    row_prev,col_new=-500,0
    for idx in nonzero_indices:
        row, col = idx
#         print(row_prev)
        if row_prev==row:
            col_new=col_new+1
        else:
            col_new=0
            row_prev=row
        query[row, col_new] = result_tensor[row, col]

    cols= target_ids.size()[1]

    return query[:, :cols]


def generate_square_subsequent_mask(dim1: int, dim2: int , device = device):

    return torch.triu(torch.ones(dim1, dim2) * float('-inf'), diagonal=1).to(device)


def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

#     print(src_seq_len,tgt_seq_len)
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len,tgt_seq_len)
#     print(f"tgt mask: {tgt_mask.size()}")
    src_mask = generate_square_subsequent_mask(tgt_seq_len,src_seq_len)
#     print(f"src mask: {src_mask.size()}")

    src_padding_mask = (src == 0).transpose(0, 1)
    tgt_padding_mask = (tgt == 0).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


class BertSeq2SeqClassifier(nn.Module):
    def __init__(self,tokenizer, decoder_num_labels, freeze_bert=False, num_decoder_layers=4, decoder_max_len=16):
        super(BertSeq2SeqClassifier, self).__init__()

        # BERT Encoder
        self.bert = BertModel.from_pretrained('bert-base-uncased', add_pooling_layer=False)
        self.bert.resize_token_embeddings(len(tokenizer))
        # # Hidden size from BERT
        hidden_size = self.bert.config.hidden_size
        self.bert_model = BertClassifier_1(tokenizer=tokenizer,freeze_bert=True)
        self.bert_model.to(device)

        self.d_in, self.d_out_kq, self.d_out_v, self.num_heads = 769, 512, 128, 24

        transformer_decoder_layer = nn.TransformerDecoderLayer(
            d_model=hidden_size,
            nhead=self.bert.config.num_attention_heads,
            dim_feedforward=self.bert.config.intermediate_size,
            activation=self.bert.config.hidden_act,
            layer_norm_eps=self.bert.config.layer_norm_eps,
            batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(
            transformer_decoder_layer,
            num_layers=num_decoder_layers
        )

        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.gelu = nn.GELU()
        self.drop = nn.Dropout(0.5)

        # Classifier for decoder
        self.classifier_decoder = nn.Linear(hidden_size, decoder_num_labels)

        # Positional Embeddings for Decoder
        self.positional_embeddings = nn.Embedding(decoder_max_len, hidden_size)



    def generate(self,decoded_outputs):
        logits_decoder = self.classifier_decoder(decoded_outputs)
        return logits_decoder

    def encode(self,input_ids, attention_mask,qty_pivot,labels):
        encoder_outputs,encoder_logits = self.bert_model(input_ids=input_ids, attention_mask=attention_mask,qty_pivot=qty_pivot,labels=labels)
        return encoder_outputs,encoder_logits


    def decode(self,input_ids,encoder_outputs,labels,encoder_logits,target_ids):
#         indices_list = find_indices_of_value(input_ids, value=102)
#         active_tags= encoder_logits.view(-1, 7)
#         flattened_tags = torch.argmax(active_tags, axis=1) # shape (batch_size * seq_len,)
#         attr_tags = torch.logical_or(flattened_tags==3,flattened_tags==4).view(-1,input_ids.size()[1]).type(torch.LongTensor).to(device).unsqueeze(2)
#         key_attribute = torch.cat((encoder_outputs.last_hidden_state,attr_tags),dim=2)
#         memory = self.relu(self.W(key_attribute))

#         query = create_query(input_ids,labels,target_ids).type(torch.LongTensor).to(device)
        # print(f"query train: {query}")
        # print(f"target train : {target_ids}")

        shifted_decoder_input_ids= torch.column_stack([30523 * torch.ones(len(target_ids)), target_ids.cpu()])[:, :-1].type(torch.LongTensor).to(device)

#         positions = torch.arange(query.size(1), device=query.device).expand(query.size(0), -1)
        positions = torch.arange(shifted_decoder_input_ids.size(1), device=shifted_decoder_input_ids.device).expand(shifted_decoder_input_ids.size(0), -1)

        pos_embeddings = self.positional_embeddings(positions)

        decoder_inputs_embedded = self.bert.embeddings.word_embeddings(shifted_decoder_input_ids) + pos_embeddings
#         decoder_inputs_embedded = self.bert.embeddings.word_embeddings(query) + pos_embeddings

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(encoder_outputs.last_hidden_state, decoder_inputs_embedded)

#         encoder_memory = apply_mask_to_encoder_hidden_state(encoder_outputs.last_hidden_state, flattened_tags.reshape(input_ids.size()), indices_list)
#         tag_centric_memory = torch.cat((encoder_memory,attr_tags),dim=2)
#         memory = self.relu(self.W(tag_centric_memory))
        memory = encoder_outputs.last_hidden_state


        decoded_outputs = self.transformer_decoder(tgt=decoder_inputs_embedded,
  #                                                    memory=encoder_outputs.last_hidden_state,
                                                    memory = memory,
                                                    tgt_mask= tgt_mask,
  #                                                    memory_mask=src_mask,
  #                                                    tgt_key_padding_mask=tgt_padding_mask,
                                                    # memory_key_padding_mask=src_boolean_mask,
                                                    tgt_is_causal=True)
        return decoded_outputs

    def forward(self, input_ids, attention_mask, qty_pivot,labels,target_ids):
        encoder_outputs,encoder_logits = self.encode(input_ids=input_ids, attention_mask=attention_mask,qty_pivot=qty_pivot,labels=labels)
        decoded_outputs = self.decode(input_ids=input_ids,labels=labels,encoder_outputs=encoder_outputs,encoder_logits=encoder_logits,target_ids=target_ids)

        return decoded_outputs,encoder_outputs,encoder_logits


# INITIATING SEQ2SEQ model
model = BertSeq2SeqClassifier(tokenizer=tokenizer,decoder_num_labels = 30524,num_decoder_layers=4,freeze_bert=True,decoder_max_len=32)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModelV2 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.embeddings.pos_tag_embeddings.weight', 'bert.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertSeq2SeqClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30524, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

# Code Execution

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import time
from datetime import timedelta
parameters = {
    "MAX_LEN":[128],
    "MAX_GRAD_NORM": [3,5],
    "TRAIN_BATCH_SIZE" : [8],
    "LEARNING_RATE": [1e-5]}


max_len_encoder,max_len_decoder,num_decoder_layers = 64,3,4
# import itertools
EPOCHS = 200
for comb in itertools.product(parameters["MAX_LEN"],
                              parameters["MAX_GRAD_NORM"],
                              parameters["TRAIN_BATCH_SIZE"],
                              parameters["LEARNING_RATE"]):
    print("Current Combination: ",comb)

    MAX_LEN,MAX_GRAD_NORM,TRAIN_BATCH_SIZE,LEARNING_RATE = comb[0],comb[1],comb[2],comb[3]

    # MAX_LEN, MAX_GRAD_NORM, TRAIN_BATCH_SIZE, LEARNING_RATE

#     train_size = 0.95
#     train_dataset = df_.sample(frac=train_size,random_state=200)
#     test_dataset = df_.drop(train_dataset.index).reset_index(drop=True)
#     train_dataset = train_dataset.reset_index(drop=True)
    train_dataset = df_
    # print("FULL Dataset: {}".format(df_.shape))
    # print("TRAIN Dataset: {}".format(train_dataset.shape))
    # print("TEST Dataset: {}".format(test_dataset.shape))
    training_set = EncoderDecoderDataset(train_dataset, tokenizer,max_len_encoder=max_len_encoder,max_len_decoder=max_len_decoder)
#     testing_set = EncoderDecoderDataset(test_dataset, tokenizer, 128,128)

    train_params = {'batch_size': TRAIN_BATCH_SIZE,
                    'shuffle': True,
                    'num_workers': 4
                    }
    ######################## WE CAN DO GRID SEARCH CV HERE ##########3##########
    training_loader = DataLoader(training_set, **train_params)
#     testing_loader = DataLoader(testing_set, **test_params)

    ###################### INITIALIZING THE MODEL FOR EACH SPLIT #####################################
    # model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
    model = BertSeq2SeqClassifier(tokenizer=tokenizer,decoder_num_labels = 30524,num_decoder_layers=num_decoder_layers,freeze_bert=True,decoder_max_len=max_len_decoder)
    model.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
#     torch.cuda.empty_cache()
    f1_inference,alpha = 0.0, 0.50
    for epoch in range(EPOCHS):
        start_time = time.time()
        print(f"Training epoch: {epoch + 1}")
        train(training_loader,alpha=alpha,MAX_GRAD_NORM=MAX_GRAD_NORM)
        end_time = time.time()
        time_difference_seconds = end_time - start_time
        time_difference = timedelta(seconds=time_difference_seconds)
        # Extract minutes and seconds
        minutes = time_difference.seconds // 60
        seconds = time_difference.seconds % 60
        # Print time difference for the epoch
        print(f"Epoch {epoch + 1}: Time taken - {minutes} minutes {seconds} seconds")

        pred_list,labels_list = inference_function_merge(df_test,max_len_encoder=max_len_encoder,max_len_decoder=max_len_decoder)

        f1_current,preds = check_F1_score(ground_truth_attr,pred_list)
        # print(f"Epoch value: {epoch}, current f1 score : {f1_current}")
        # print(f"Preds: {pred_list}")
#         print(f"Labels: {labels_list}")
        if epoch>10 and f1_inference<f1_current:
            ### SAVE MODEL
            f1_inference=f1_current
            df = pd.DataFrame(zip(preds,ground_truth_attr,labels_list), columns=['preds','ground_truth','Tags'])
            df.to_csv(f"/kaggle/working/f1={f1_current}-alpha={alpha}-non_tagcentric-deco{num_decoder_layers}--batchsize-{TRAIN_BATCH_SIZE}-epoch-{epoch}-maxlen-{max_len_decoder}.csv",index=False)
            # print(f"Epoch value: {epoch}, current f1 score : {f1_current} \n")
# print("\n############### COMBINATION DONE #################")
    print("\n############### COMBINATION DONE #################")